In [ ]:
%% Load data
cat = imread('IMG_5089.JPG');
cat_grey = rgb2gray(cat);

flower = imread('IMG_4096.JPG');
flower_grey = rgb2gray(flower);

err_cat = zeros(size(cat_grey,1)/65,size(cat_grey,2)/64);

%% process image with noise
[cat_block, cat_noise] = ImgBlockPartition(cat_grey,65,64,0.5);
[M,N,P] = size(cat_block);
cat_noise = reshape(cat_noise,M,N*P);
cat_block = reshape(cat_block,M,N*P);

[D,Xh] = kSVD(cat_noise,300,20); % M, and K to be determined
cat_hat = D*Xh;
for kk = 1:size(Xh,2)
    err_cat = sqrt(vnorm(cat_block(:,kk)-cat_hat(:,kk),'fro').^2./(65*64));
end

cat_reconstruct = reshape(cat_hat(:),size(cat_grey));
figure(1)
imshow(cat_reconstruct)


%% function
function [img_block,noise_block] = ImgBlockPartition(img,row,col,noise)
% partition the pixels into blocks
% syntax img_block = ImgBlockPartition(img,row,col)
% INPUT : img         - image in grey scale
%         row         - size of row of the image block
%         col         - size of column of the image block
%         noise       - noise level, from 0.1 to 0.9
% OUTPUT: img_block   - block output of the image block matrix 
%                     (row x col x [num blocks in row] x [num blocks in col])
%         noise_block - block output of the image block matrix with noise
%                     (row x col x [num blocks in row] x [num blocks in col])
[M,N] = size(img);
img_block = zeros(row*col,M/row,N/col);
noise_block = zeros(row*col,M/row,N/col);
for ii = 1:M/row
    tempseg = img((ii*row-(row-1)):ii*row,:);
    for jj = 1:N/col
        tempblock= tempseg(:,(jj*col-(col-1)):jj*col);
        img_block(:,ii,jj)= tempblock(:);
        noise_block(:,ii,jj) = tempblock(:);
        randind = sort(randperm(row*col,round(row*col*noise)));
        %[I,J] = ind2sub([row,col],randind);
        noise_block(randind,ii,jj) = 0;
    end
end
%img_block = reshape(img_block,row*col,(M/row)*(N/col));
end


